# Bag of Words and TFIDF Analysis of 20 Spotify Songs


### Making sure that additional python packages are installed

In [0]:
!pip install spotipy --upgrade

In [0]:
!pip install lyricsgenius

     |████████████████████████████████| 92kB 3.0MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [0]:
!pip install nltk

### Setting up env variables

In [0]:
%env SPOTIPY_CLIENT_ID=50909c91e8284e55ac36382563898444
%env SPOTIPY_CLIENT_SECRET=e0e60cd6d17342c5a2c183c2eeebd6a3
%env GENIUS_CLIENT_ID=CM-Twc16fd2BWpO2KXoTccWJE1yFOSbUGOMUiBDBv4dDPRpVJLDjccpvhpyIDMAB
%env GENIUS_CLIENT_SECRET=JJzWYYY38d5fEAcYzJa9ZjHV2b1d-u5SVk7uBNvEyJlSJ9PK_5Jx-rBY9dnCCEiSwOWHzfABp3h_iOMC6wTBPg
%env GENIUS_CLIENT_ACCESS_TOKEN=4JzQ7FOp8JOUXpH5_7IJ-89lccqK8H-tfrTd5WGfOaiiK3dUibOFzSS5dvFOdK28

env: SPOTIPY_CLIENT_ID=50909c91e8284e55ac36382563898444
env: SPOTIPY_CLIENT_SECRET=e0e60cd6d17342c5a2c183c2eeebd6a3
env: GENIUS_CLIENT_ID=CM-Twc16fd2BWpO2KXoTccWJE1yFOSbUGOMUiBDBv4dDPRpVJLDjccpvhpyIDMAB
env: GENIUS_CLIENT_SECRET=JJzWYYY38d5fEAcYzJa9ZjHV2b1d-u5SVk7uBNvEyJlSJ9PK_5Jx-rBY9dnCCEiSwOWHzfABp3h_iOMC6wTBPg
env: GENIUS_CLIENT_ACCESS_TOKEN=4JzQ7FOp8JOUXpH5_7IJ-89lccqK8H-tfrTd5WGfOaiiK3dUibOFzSS5dvFOdK28


### Import needed libraries

In [0]:
import os
import re
import math
from pprint import pprint

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import lyricsgenius

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

import heapq
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# 1

## Get the contents of *CS 174 - Sample Playlist* playlist 

In [0]:
# Chill Hits: spotify:playlist:37i9dQZF1DX4WYpdgoIcn6
# Broken Heart: spotify:playlist:37i9dQZF1DXbrUpGvoi3TS
# CS 174 - Sample Playlist: spotify:playlist:3It9QKvtI4QHdF3ffVbuGQ
playlist_uri = 'spotify:playlist:3It9QKvtI4QHdF3ffVbuGQ'
sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

results = sp.playlist_tracks(playlist_uri, limit=20, fields="items.track.id,items.track.name,items.track.artists")

song_list = {index: results['items'][index]['track']['name'] for index in range(len(results['items']))}
song_artist_dict = {item['track']['name']:item['track']['artists'][0]['name'] for item in results['items']}

pprint(song_list)

{0: 'Be My Mistake',
 1: 'Falling',
 2: 'i love you',
 3: "I Don't Wanna Love You Anymore",
 4: 'Why',
 5: 'Someone New',
 6: 'N Y L A',
 7: "when the party's over",
 8: 'Dancing With Your Ghost',
 9: "Don't Watch Me Cry",
 10: 'Julia',
 11: 'If the World Was Ending - feat. Julia Michaels',
 12: 'Lonely',
 13: "i don't think i love you anymore",
 14: 'Sorry Would Go A Long Way',
 15: "It's You",
 16: 'In Vain',
 17: 'Same Room',
 18: 'Six Feet Under',
 19: 'hope is a dangerous thing for a woman like me to have - but I have it'}


# 2

In [0]:
genius = lyricsgenius.Genius(os.environ["GENIUS_CLIENT_ACCESS_TOKEN"])

lyrics_list = {}
for index, song in song_list.items():
  song_lyrics = genius.search_song(song, song_artist_dict[song]).lyrics
  song_lyrics = song_lyrics.replace('\u2005', ' ')
  song_lyrics = song_lyrics.split('\n')
  lyrics_list[index] = song_lyrics

pprint(lyrics_list)

Searching for "Be My Mistake" by The 1975...
Done.
Searching for "Falling" by Harry Styles...
Done.
Searching for "i love you" by Billie Eilish...
Done.
Searching for "I Don't Wanna Love You Anymore" by LANY...
Done.
Searching for "Why" by Bazzi...
Done.
Searching for "Someone New" by BANKS...
Done.
Searching for "N Y L A" by blackbear...
Done.
Searching for "when the party's over" by Billie Eilish...
Done.
Searching for "Dancing With Your Ghost" by Sasha Sloan...
Done.
Searching for "Don't Watch Me Cry" by Jorja Smith...
Done.
Searching for "Julia" by Lauv...
Done.
Searching for "If the World Was Ending - feat. Julia Michaels" by JP Saxe...
Done.
Searching for "Lonely" by Noah Cyrus...
Done.
Searching for "i don't think i love you anymore" by Alaina Castillo...
Done.
Searching for "Sorry Would Go A Long Way" by Tori Kelly...
Done.
Searching for "It's You" by Ali Gatie...
Done.
Searching for "In Vain" by Sigrid...
Done.
Searching for "Same Room" by JP Saxe...
Done.
Searching for "Six F

# 3 (Bag of Words)

## Preprocessing (convert to lowercase, remove non-alphanumeric characters and stop words)

In [0]:
stop_words = [re.sub(r'[^a-z0-9 ]+', '', word) for word in set(stopwords.words('english'))]

filtered_lyrics = {}
for index in lyrics_list.keys():
  lyrics = lyrics_list[index]
  filtered_lyrics[index] = []
  for line in lyrics:
    if not line or line[0] == '[':
      continue
    line = re.sub(r'[^a-z0-9 ]+', '', line.lower())
    parsed_corpus = line.split()
    filtered = [word for word in parsed_corpus if word not in stop_words]
    filtered_lyrics[index] += filtered
    
pprint(filtered_lyrics) #tor's notes: not yet tokenized! Tokenized per line
#kyle's change: transformed 3d list to 2d

{0: ['mistake',
     'turn',
     'light',
     'bought',
     'jeans',
     'ones',
     'like',
     'want',
     'hug',
     'want',
     'sleep',
     'smell',
     'hair',
     'reminds',
     'feet',
     'wait',
     'outside',
     'hotel',
     'room',
     'wait',
     'till',
     'give',
     'sign',
     'cause',
     'get',
     'lonesome',
     'sometimes',
     'save',
     'jokes',
     'going',
     'make',
     'whilst',
     'see',
     'much',
     'drink',
     'take',
     'mistake',
     'called',
     'cause',
     'speak',
     'make',
     'hard',
     'makes',
     'weak',
     'wait',
     'outside',
     'hotel',
     'room',
     'wait',
     'till',
     'give',
     'sign',
     'cause',
     'get',
     'lonesome',
     'sometimes',
     'save',
     'jokes',
     'going',
     'make',
     'whilst',
     'see',
     'much',
     'drink',
     'take',
     'mistake'],
 1: ['im',
     'bed',
     'theres',
     'one',
     'blame',
     'drink',
     'w

In [0]:
#create a word frequency frequency dictionary [words] = frequency
wordfreq = {}
for index in filtered_lyrics.keys():
  for word in filtered_lyrics[index]:
      if word not in wordfreq.keys():
        wordfreq[word] = 1
      else:
        wordfreq[word] += 1

wordfreq = {k: v for k, v in sorted(wordfreq.items(), key=lambda item: item[1])}
pprint(wordfreq)

{'15year': 1,
 '2014': 1,
 '247': 2,
 '2800': 1,
 'aarons': 1,
 'accent': 1,
 'admit': 1,
 'afford': 1,
 'ago': 1,
 'ah': 1,
 'aint': 4,
 'air': 1,
 'alarms': 1,
 'alone': 4,
 'along': 2,
 'already': 3,
 'alright': 5,
 'also': 1,
 'always': 6,
 'another': 1,
 'anymore': 25,
 'anyone': 5,
 'anything': 5,
 'apart': 5,
 'argue': 1,
 'arms': 2,
 'around': 5,
 'ashamed': 1,
 'ask': 1,
 'asleep': 2,
 'ass': 1,
 'asshole': 1,
 'assuming': 1,
 'awake': 3,
 'aware': 3,
 'away': 15,
 'baby': 41,
 'back': 25,
 'bad': 2,
 'baggage': 1,
 'basement': 1,
 'bathroom': 3,
 'beachwood': 1,
 'became': 2,
 'bed': 5,
 'bedroom': 1,
 'believe': 6,
 'best': 6,
 'better': 5,
 'beyond': 1,
 'black': 1,
 'blame': 5,
 'bleed': 2,
 'bleedin': 1,
 'blood': 2,
 'bloom': 6,
 'blow': 1,
 'born': 1,
 'bought': 1,
 'bout': 1,
 'bowery': 1,
 'boy': 1,
 'break': 9,
 'breakup': 1,
 'breath': 2,
 'breathe': 1,
 'bring': 4,
 'broken': 8,
 'bruised': 1,
 'bums': 1,
 'burn': 1,
 'burned': 2,
 'burns': 5,
 'burnt': 1,
 'cafe':

In [0]:
freq_words = heapq.nlargest(100, wordfreq, key=wordfreq.get)
print(freq_words)

['im', 'love', 'know', 'give', 'like', 'baby', 'say', 'cause', 'come', 'want', 'make', 'cant', 'ill', 'back', 'anymore', 'oh', 'never', 'stay', 'ive', 'got', 'heart', 'right', 'night', 'wanna', 'yeah', 'one', 'would', 'still', 'please', 'even', 'get', 'someone', 'falling', 'tell', 'away', 'see', 'could', 'trust', 'think', 'new', 'time', 'vain', 'let', 'way', 'day', 'go', 'world', 'room', 'left', 'call', 'feel', 'ending', 'lie', 'leave', 'gonna', 'thing', 'york', 'care', 'ooh', 'try', 'hope', 'la', 'til', 'every', 'help', 'break', 'woman', 'wait', 'talk', 'crying', 'fall', 'past', 'broken', 'hurt', 'theres', 'meant', 'feeling', 'need', 'maybe', 'sick', 'id', 'really', 'everything', 'promise', 'ever', 'dancing', 'ghost', 'wonder', 'lonely', 'dangerous', 'sometimes', 'much', 'take', 'fallin', 'mean', 'wish', 'cry', 'us', 'mind', 'thought']


In [0]:
song_vectors = []
for index in filtered_lyrics.keys():
  song_vec = []
  for token in freq_words:
    if token in filtered_lyrics[index]:
      song_vec.append(1)
    else:
      song_vec.append(0)
  song_vectors.append(song_vec)

print(song_vectors)
len(song_vectors[0])

[[0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0], [1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0], [1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0], [0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1,

100

In [0]:
BoW_matrix = [[0 for row in filtered_lyrics.keys()] for column in filtered_lyrics.keys()] #initializing matrix
pd.set_option('display.max_columns', None)

for row in filtered_lyrics.keys():
  for column in filtered_lyrics.keys():
    BoW_matrix[row][column] = float(cosine_similarity([song_vectors[row]], [song_vectors[column]]))
print(pd.DataFrame(BoW_matrix)) #converted into a dataframe to present the matrix nicely

           0         1         2         3         4         5         6  \
0   1.000000  0.251976  0.263523  0.163663  0.132453  0.184900  0.136083   
1   0.251976  1.000000  0.358569  0.412393  0.300376  0.349428  0.257172   
2   0.263523  0.358569  1.000000  0.310530  0.376969  0.438529  0.172133   
3   0.163663  0.412393  0.310530  1.000000  0.346844  0.302614  0.133631   
4   0.132453  0.300376  0.376969  0.346844  1.000000  0.404095  0.108148   
5   0.184900  0.349428  0.438529  0.302614  0.404095  1.000000  0.377426   
6   0.136083  0.257172  0.172133  0.133631  0.108148  0.377426  1.000000   
7   0.204124  0.257172  0.387298  0.311805  0.378517  0.415168  0.277778   
8   0.125988  0.142857  0.318728  0.288675  0.300376  0.314485  0.257172   
9   0.277778  0.419961  0.351364  0.363696  0.397360  0.369800  0.272166   
10  0.176777  0.356348  0.298142  0.424334  0.421464  0.359546  0.144338   
11  0.115470  0.174574  0.292119  0.302372  0.183533  0.352282  0.282843   
12  0.277778

# 4 (TFIDF)

In [0]:
unique = wordfreq.keys()

numwords_dict_list = []

#creating wordfreq for each song
for index in filtered_lyrics.keys():
  numwords = dict.fromkeys(unique, 0)
  for word in filtered_lyrics[index]:
    numwords[word] += 1
  numwords_dict_list.append(numwords)

pprint(numwords_dict_list[0])

{'15year': 0,
 '2014': 0,
 '247': 0,
 '2800': 0,
 'aarons': 0,
 'accent': 0,
 'admit': 0,
 'afford': 0,
 'ago': 0,
 'ah': 0,
 'aint': 0,
 'air': 0,
 'alarms': 0,
 'alone': 0,
 'along': 0,
 'already': 0,
 'alright': 0,
 'also': 0,
 'always': 0,
 'another': 0,
 'anymore': 0,
 'anyone': 0,
 'anything': 0,
 'apart': 0,
 'argue': 0,
 'arms': 0,
 'around': 0,
 'ashamed': 0,
 'ask': 0,
 'asleep': 0,
 'ass': 0,
 'asshole': 0,
 'assuming': 0,
 'awake': 0,
 'aware': 0,
 'away': 0,
 'baby': 0,
 'back': 0,
 'bad': 0,
 'baggage': 0,
 'basement': 0,
 'bathroom': 0,
 'beachwood': 0,
 'became': 0,
 'bed': 0,
 'bedroom': 0,
 'believe': 0,
 'best': 0,
 'better': 0,
 'beyond': 0,
 'black': 0,
 'blame': 0,
 'bleed': 0,
 'bleedin': 0,
 'blood': 0,
 'bloom': 0,
 'blow': 0,
 'born': 0,
 'bought': 1,
 'bout': 0,
 'bowery': 0,
 'boy': 0,
 'break': 0,
 'breakup': 0,
 'breath': 0,
 'breathe': 0,
 'bring': 0,
 'broken': 0,
 'bruised': 0,
 'bums': 0,
 'burn': 0,
 'burned': 0,
 'burns': 0,
 'burnt': 0,
 'cafe': 0,


In [0]:
def computeTF(dictionary, bow):
  tf_dict = {}
  bow_count = len(bow)
  for word, count in dictionary.items():
    tf_dict[word] = count / float(bow_count)
  return tf_dict
  

tf_dict_list = []
for index in filtered_lyrics.keys():
  tf_dict_list.append(computeTF(numwords_list[index], filtered_lyrics[index]))

pprint(tf_dict_list[0])  

NameError: ignored

In [0]:
def computeIDF(docs):
  idf_dict = {}

  #number of documents
  n = len(docs)

  #counts the number of documents that contain a word w
  idf_dict =  dict.fromkeys(docs[0].keys(), 0)
  for doc in docs:
    for word, val in doc.items():
      if val > 0:
        idf_dict[word] += 1

  #divide by n by denominator above, take the log of that
  for word, val in idf_dict.items():
    idf_dict[word] = math.log(n / float(val))

  return idf_dict


idfs = computeIDF(numwords_dict_list)

In [0]:
def tfidf(tf_bow, idfs):
  tfidf = {}
  for word, val in tf_bow.items():
    tfidf[word] = val * idfs[word]
  return tfidf

tfidf_dict_list = []

for index in filtered_lyrics.keys():
  tfidf_dict_list.append(tfidf(tf_dict_list[index], idfs))

pprint(pd.DataFrame(tfidf_dict_list))  

Creating Word2Vec Model

In [0]:
model = Word2Vec(size=100,window=5,min_count=30, sg=0, alpha = 0.025)

In [0]:
tfidf_matrix = [[0 for row in filtered_lyrics.keys()] for column in filtered_lyrics.keys()]
pd.set_option('display.max_columns', None)

for row in filtered_lyrics.keys():
  for colum in filtered_lyrics.keys():
    tfidf_matrix[row][column] = float(cosine_similarity([tfidf_list[row]], [tfidf_list[column]]))
print(pd.DataGrame(tfidf_matrix))

# 5 (Analysis with Graphs)

In [0]:
import seaborn as sns
%matplotlib inline

df = pd.DataFrame(BoW_matrix)
plt.figure(figsize = (20,10))
sns.heatmap(df, center=0.5, cmap="coolwarm", annot=True)

# 6 (Analysis of top 5 words and representation)